# Data analysis with Python — PART 3

## Grouping and Merging (and a few formulas)

You've learned how to import files, filter and sort. Now you're going to learn how to aggregate, use formulas and merge tables. 

<b>The objective: Find which gubernatorial candidate won each precinct, and if any majority-Republican precincts voted for Katie Hobbs, the democrat candidate.</b> 

Precincts can be interesting to see how certain areas of the county swung. Specifically, we've used precinct-level data to identify areas that had [voted for Trump in 2016 but flipped for Biden in 2020.](https://www.azcentral.com/story/news/politics/elections/2020/11/18/how-did-president-elect-joe-biden-win-arizona-map-maricopa-county-votes-reveals-one-key-path-victory/6328880002/) Or, more recently, we've used precinct-level data to identify areas that are [Republican strongholds that didn't come out for Kari Lake in the numbers that she needed.](https://www.azcentral.com/story/news/politics/elections/2022/11/23/republican-voters-fueled-arizona-democratic-wins-in-midterms/69671685007/) 

But as you've probably come to realize, the structure of our election data doesn't allow us to easily see who won which race in each precinct, and what percent of the vote each candidate got. Our election file also doesn't have any party registration data in it. 

Start by importing pandas and the elections file.

In [94]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('elections_03_02_23.csv',
                 dtype={'i_contest_id': str, 'candidate_id': str})                 

# Clean the data first 

The county excludes certain candidate types when it is calculating the total votes per race. By selecting for only candidates with an "R" OR "Q" designation in the candidate_type field, we are following their method for including only named candidates and qualified write-ins. 

The filter below overwrites our dataframe — df — so that it only included either Q candidates OR R candidates. The pipe symbol ( | ) means "or." 

In [72]:
df = df[(df['candidate_type'] == 'Q') | (df['candidate_type'] == 'R')].copy()

And remember those precincts with zero voters? Let's get rid of those as well. Write that filter below.

In [73]:
df = 

## Select the Govenor Race 
Write a filter that creates a table that includes only the Governor contest. I've called it "gov" here.

In [76]:
gov = 

## Calculate the total votes per precinct

The winner per precinct is going to be calculated by dividing a candidate's votes over the total number of votes cast in that contest. To get the total number of votes cast, we'll need to use * drumroll * a GroupBy.

Think of a GroupBy like a pivot table in Excel. In Excel, you drag the column that you want to group by into rows, and the column that you want to measure into Values.

With pandas, you'll be dropping the column you want to group by in parentheses, and the column you want to measure into brackets, followed by the math you want to do on that column.  

The example below sums votes for each precinct.

If for whatever reason you wanted to get an average number of votes each precinct reported per contest, you would write .mean() instead. If you wanted to get the maximum number of votes any precinct reported for each contest, you'd write .max(). Same idea for .min() 

The main idea here is that you can use a groupby to do a wide variety of math things on any given column and the syntax is almost always going to be the same.

In [96]:
gov_total = gov.groupby('precinct_name')['votes'].sum().to_frame(name = 'total_votes').reset_index()
gov_total.head()

OK, but what's all the stuff after the .sum() command? 

.to_frame(name = 'total_votes') is optional. This is simply here if I wish to rename my column header in my groupby. If you run the groupby command followed only by .reset_index(), the computer will retain your original column header. And maybe that's fine, and you want that. But sometimes you won't want that. In this case, I want to make the distinction that my vote column is a total of all votes, so I'm being extra specific. 

.reset_index() is not optional, if you have any hope to use your groupby again. Simply running the groupby command without results in a print out that you cannot use as a table. Run the code below and you'll see what I mean. 

Resetting your index allows the computer to make a new table that you can call upon just like when you call upon the original dataframe. 

In [97]:
test = gov.groupby('precinct_name')['votes'].sum()
test.head()

## Isolate only columns we care about

This database is massive. There are a lot of columns that we don't need that might just end up confusing us. If our objective is to find out how many votes each candidate got relative to the total votes, lets' just grab the bare minimum we need to do achieve that. 



In [98]:
#create a subset of the original dataframe that just contains candidate_name, votes and precinct_name
#any time you want to create a truncated version of your dataset, you can write the name of the table you want to truncate,
#follow with brackets, and then put your desired column names into single or double quotes. 
#In this example I have named my new table gov_cand_votes

#Just run this code
gov_cand_votes = gov[['candidate_name', 'precinct_name', 'votes']]
gov_cand_votes

## Merging
Ok. So we have two separate dataframes. One is our groupby that sums the total votes per precinct. The other is a slice of our original dataframe that tells us how many votes each candidate got. How do we calculate percentages? These tables alone aren't super helpful. But combining them is how we answer one of our questions. 

Enter: Merging. 

Let me first start by stressing that there are various ways to merge different datasets. In the future, for example, you may find yourself needing to merge data based on location values (like lat/long) — that's called a spatial join, and it requires a different toolkit and approach than what we'll be using today.

Today, we're showing you how to merge on a common column. 

([Here's a little illustration explaining what, exactly, we're doing](https://jamboard.google.com/d/1P3EDbX9YlOHig6CtDNhKISTAKcf8yCc8F5YT0D-xK0E/edit?usp=sharing))

Our two datasets have a common column: precinct_name. 

In order for this to work, the matching columns in both datasets need to be the same datatype. If precinct_name were a string in one file and an integer in the other, we would get an error message. (This is one of the reasons why it's important to specify dtypes upon import.) 

You have several options for column-matching merges, depending on your desired outcome. [This page gives a good explanation of left, right, inner and outer merges.](https://www.analyticsvidhya.com/blog/2020/02/joins-in-pandas-master-the-different-types-of-joins-in-python/#:~:text=Full%20Join%2C%20also%20known%20as,that%20lacks%20a%20matching%20row.) 

We're going to do an inner merge, which just means that only values contained in both datasets will survive the merge. If I have precinct A in one file, but it doesn't appear in the other, then the file resulting from my merge will not have precinct A in it. 

(If I wanted to keep precinct A regardless of its appearance in the other dataset, I'd use a left or right merge. Refer to the page I linked to about this.) 

Here's the syntax for an inner merge where the column names that you're merging on are the same — it's a nice, short line:

    output_data = pd.merge(table_1,table_2)

If your column names were different (or you wanted to do a left or a right merge), you'd need a little more:

    output_data = table_1.merge(table_2, left_on='table_1_column', 
    right_on='table_2_column',how="inner")

Here's how this works:
-  Your left dataset is considered the one to the left of .merge 
-  Your right dataset is considered the one to the right of the (
-  *left_on=''* will correspond to the merge column name in your left dataset
-  *right_on=''* will correspond to the merge column name on the right.
-  *how='inner'* is the type of join we want. If you wanted to keep everything in the left dataset, regardless if it appeared in the right, you'd write 'left' there.

Merges are complicated. It's totally okay if you're like "left? right? huh????" right now! We're happy to answer any questions you might have about merges. 

Try to write a merge on your own that brings together our two tables, using either of those two sample code lines above. 


In [99]:
#I'm calling your merge plv, short for precinct-level votes
plv = 
plv

You should now have in the same table the number of votes each candidate got in each precinct AND the total votes for the Govenor race coming from that precinct. So you could do the math to determine the percent of votes. We'll do that next. 

# Determining winners

Now it's time to expand that table and make the computer do some math for us! We'll be adding columns to our table. Any time you want to create a new column, you will name the the dataframe that you're using, open brackets, and put your desired column name in quotes in those brackets. 

Follow it with what you want to be in that column. In this case, we are saying that our new column = votes divided by total votes. Yes, you have to specify each time which dataframe you're dealing with. 

The code below assumes you named named your summed votes from your groupby above 'total_votes'. If you called it something else, replace 'total_votes' with whatever you called it.


In [115]:
plv['perc_of_votes'] = plv['votes']/plv['total_votes']
plv.head()

But we still can't see who won each precinct, right? We'd have to eyeball it. Let's do one more groupby - but this time we're going to drop the results of the groupby into a column within our database. 

This will look a little different from our first two group bys.

Instead of creating a new dataframe for this groupby, we're creating a new column within our preexisting dataframe. We do that just as we did above: write the name of the dataframe you want to drop your new column, and the name of your new column. (In brackets, in quotes.) 

To determine first place, second place, third place, etc. we will use .rank(). 

In [103]:
plv["rank"] = plv.groupby("precinct_name")["votes"].rank(method="dense", ascending=False)
plv.head()

This code groups precincts and then calculates each vote tally's place in that precinct. Because we specified ascending = False, our largest vote tally will be given a 1.

...bUt wHaT aBoUt TiEs? 

If you are wondering that, good for you. Though it seems unlikely the exact same number of people in one precinct came out for two candidates, it happens. And if you don't account for it - your results will be messed up. 

method=dense means that the computer will assign the same rank to ties. So if the top two vote tallies are a tie, they will both have a 1. 

## Labeling winners

Now, let's filter only for rows where the rank is 1. If there are no ties, we should have only one row per precinct. So let's follow up our filter with a groupby that counts the number of rows there are per precinct, and plunks the answer into a new column. 

In [33]:
#select only rows where rank is 1
winners = plv[plv['rank'] == 1].copy()

In [105]:
#count rows per precinct
winners['rows_per_precinct'] = winners.groupby('precinct_name')['precinct_name'].transform('count')
winners.head()

...hold on, what is .transform? Why does this groupby look a little different than the others? 

Most of the time when you plunk the results of a groupby into an existing table as a new column, you'll need to use .transform(). You put the math function you want to do inside those parentheses - with quotes around it, just like you see here with 'count'.

Finally, we will sort that new row so we can see any instances where there are more than 1 at the top.

In [106]:
#sort largest to smallest row count
winners.sort_values('rows_per_precinct', ascending=False)

We have one tie. 

So now let's add a winner column. 

Enter the where statement. This might look something like an if/then statement if you've used those in Excel. Here, we are telling the computer that for every row rows_per_precint = 1, to place the corresponding candidate_name value in a new column called "winner." If there is anything other than a 1 in the rows_per_precinct column, we're telling the computer to put "TIE" in that winner column.

Pandas doesn't do where statements, but a library called numpy does. Just like we imported skimpy to do a little work outside the pandas wheelhouse, we'll import numpy here. 

In [108]:
#for where statements, you need a library called numpy
import numpy as np

winners['winner'] = np.where((winners['rows_per_precinct'] == 1), (winners['candidate_name']), "TIE")
winners.sort_values('rows_per_precinct', ascending=False)

## Cleaning our data before we attach voter registration information

To format this data so we don't have double rows for ties, you could drop the only column that contains differing values and then drop duplicates. Notice how, for our Pointe Precinct, candidate_name is the only column that makes the two rows different?   

In [110]:
winners = winners.drop(columns =['candidate_name'])
winners = winners.drop_duplicates()

#write a sort by rows_per_precinct to make sure we have indeed gotten rid of our double precinct


## Bringing in the context

Kari Lake lost her bid to be Arizona's governor because too many Republicans didn't like her. One way we can see that is by identifying areas that swung for Katie Hobbs despite having more registered Republicans than anything else.

Follow the steps below to see if you can find out how many Republican-majority precincts went for Katie Hobbs. If you're feeling extra ambitious, see if you can find out how many Dem-majority precincts went for Kari Lake. 


In [111]:
#import the voter_reg file
voter_reg =

In [89]:
#view the first five rows of your data


In [114]:
#merge the voter_reg table to the winners table
gov_race = 
gov_race

In [92]:
#filter for majority Republican precincts
rep = 

In [113]:
#group by the winner column, and count how many precincts went for Hobbs and how many went for Kari
my_groupby = 

## Congrats! 
You've learned some Python data analysis fundementals. We hope you will continue to practice Python on the job and reach out to us if you have any questions! 

cmcglade@gannett.com

sjayaraman@gannett.com